<center><h1>Where is Elon?!</h1></center>

Elon musk's plane tracker, will include:
- Data scraping
- Data visualization

Jet information
- Registration: N628TS
- ICAO hex number: A835AF
- Category: A3 Large (75000 to 300000 lb)
- Bought in 2006

Source is ADS-B exchange (All proper APIs are priced)
- https://globe.adsbexchange.com
- https://globe.adsbexchange.com/?icao=a835af&lat=37.731&lon=-122.215&zoom=1.0&showTrace=2023-02-08 Using their replay function for day -> showTrace=YYYY-MM-DD
- https://globe.adsbexchange.com/?replay=2023-02-16-11:46&lat=40.560&lon=-73.660&zoom=9.0 (Deprecated) Using their replay function -> replay=YYYY-MM-DD-HH:MM

- Beginning of data seems to be on 2020-01-19

Settings:
- replay=YYYY-MM-DD-HH:MM
- icao=a835af
- lat=0
- lon=0
- zoom=2.0

Javascript variable:
- selectedPlanes
    - fullTrace

<!-- 
other info (temporary)
ads-b exchange replay function
https://globe.adsbexchange.com/?replay=2023-02-16-11:46&lat=40.560&lon=-73.660&zoom=9.0


maybe:
https://globe.adsbexchange.com/?replay=2023-03-01&icao=a835af


https://globe.adsbexchange.com/?icao=a835af&lat=37.731&lon=-122.215&zoom=1.0&showTrace=2023-02-08 <- we got a winner!!!!!

console:
selectedPlanes()
-> fullTrace
-> trace -->

<h2>Imports and settings</h2>

In [8]:
from selenium import webdriver

import json
import pandas as pd

from time import sleep
import sys

from datetime import datetime, timedelta
from math import floor

In [7]:
#Scraper settings
targetIcao = "a835af"
startDate = datetime.strptime("2020/01/19", '%Y/%m/%d').date()
endDate = datetime.now().date() # datetime.strptime("2023/02/08", '%Y/%m/%d').date()

lat = "0.000"
lon = "0.000"
zoom = "2.0"

#Saving settings
save_file = "tracking_data.json"

In [11]:
#Other data
javaScript_start_time = datetime.strptime("1970/01/01", '%Y/%m/%d') #Time delta in seconds

<h2>Scrap data</h2>

Data from - Selected Planes() -> showTrace

Problem -> Circular reference in javascript object so can't return it </n>
Solution -> https://stackoverflow.com/a/10455320/454773 get rid of circular reference + parse to json

Trace data as:
- list of values + dict
    - [<b>time</b> (javscript time), <b>latitude</b>, <b>longitude</b>, <b>altitude/'ground'</b>, <b>ground speed (kt)</b>, ground track, ?, ?, <b>dict</b>, transponder type (I think), geometric altitude, ?, indicated speed, ?]
    - dict -> keys depend on ground or air + sometimes no info
        - Gonna take: 'flight' (flight number), 'alt_geom' (geometric altitude), 'ias' (indicated speed), 'tas' (true speed), 'mach', 'wd' (wind direction), 'ws' (wind speed), 'track' (ground track), 'mag_heading' (magnetic heading), 'true_heading', 'baro_rate' (barometric climb (ft/min)), 'geom_rate' (rate of climb or decent ft/min)

ground track -> aircraft direction above ground (?)
geometric altitude -> The geometric GPS derived altitude referenced to WGS84 ellipsoid <!-- (or reference EGM96 when enabled in the settings) -->

Data Storage -> Saved in dictionary
- key -> date time information
- value -> dict {lat, lon, baro_alt, grd_speed, extra : dict2}
    - dict2 -> dict {flight_num, geom_alt, geom_rate, baro_rate, indicated_spd, true_spd, mach, wind_dir, wind_spd, grd_track, mag_heading, true_heading} (Not always complete)

In [12]:
#Utility functions
util_bld_dict2_ins = ["flight", "alt_geom", "geom_rate", "baro_rate", "ias", "tas", "mach", "wd", "ws", "track", "mag_heading", "true_heading"]
util_bld_dict2_outs = ["flight_num", "geom_alt", "geom_rate", "baro_rate", "indicated_spd", "true_spd", "mach", "wind_dir", "wind_spd", "grd_track", "mag_heading", "true_heading"]

def build_dict(lin):
    info = {}
    info["lat"] = lin[1] if len(lin) > 1 else None
    info["lon"] = lin[2] if len(lin) > 2 else None
    info["baro_alt"] = lin[3] if len(lin) > 3 else None
    info["grd_speed"] = lin[4] if len(lin) > 4 else None
    if len(lin) < 9 or lin[8] == None:
        info["extra"] = None
    else:
        extra = {}
        for i in range(len(util_bld_dict2_ins)):
            v = lin[8].get(util_bld_dict2_ins[i])
            if v != None:
                extra[util_bld_dict2_outs[i]] = v
        info["extra"] = extra
    return info

def get_java_time(dt):
    return datetime.strptime("1970/01/01", '%Y/%m/%d') + timedelta(seconds=floor(dt))

def has_trace(obj):
    return len(obj) > 0 and obj[0].get("fullTrace") != None and obj[0]["fullTrace"].get("trace") != None

In [13]:
import sys

def progressbar(p, prefix="", size=60, out=sys.stdout): # Python3.3+
    x = int(size*p)
    print("{}[{}{}] {}%".format(prefix, "#"*x, "."*(size-x), int(p*100)), end='\r', file=out, flush=True)

In [14]:
driver = webdriver.Chrome()

jss = '''const getCircularReplacer = () => {
  const seen = new WeakSet();
  return (key, value) => {
    if (typeof value === "object" && value !== null) {
      if (seen.has(value)) {
        return;
      }
      seen.add(value);
    }
    return value;
  };
};

return JSON.stringify(selectedPlanes(), getCircularReplacer());
'''

#Declare data storage
data = {}

delta = endDate - startDate
for i in range(delta.days + 1):
    #Get webpage
    driver.get("https://globe.adsbexchange.com/?icao={}&lat={}&lon={}&zoom={}&showTrace={}".format(targetIcao, lat, lon, zoom, startDate + timedelta(days=i)))

    #Give enough time for javascript execution + information return
    c = 0
    while (c < 5):
      sleep(1)
      js_data = json.loads(driver.execute_script(jss))
      c += 10 if has_trace(js_data) else 1

    if(has_trace(js_data)):

        for l in js_data[0]['fullTrace']['trace']:
            t = get_java_time(l[0])
            data[str(t)] = build_dict(l)
    
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x00B737D3]
	(No symbol) [0x00B08B81]
	(No symbol) [0x00A0B36D]
	(No symbol) [0x009EE5DA]
	(No symbol) [0x00A5F0CB]
	(No symbol) [0x00A70A56]
	(No symbol) [0x00A5B216]
	(No symbol) [0x00A30D97]
	(No symbol) [0x00A3253D]
	GetHandleVerifier [0x00DEABF2+2510930]
	GetHandleVerifier [0x00E18EC1+2700065]
	GetHandleVerifier [0x00E1C86C+2714828]
	GetHandleVerifier [0x00C23480+645344]
	(No symbol) [0x00B10FD2]
	(No symbol) [0x00B16C68]
	(No symbol) [0x00B16D4B]
	(No symbol) [0x00B20D6B]
	BaseThreadInitThunk [0x75727D69+25]
	RtlInitializeExceptionChain [0x7710BB9B+107]
	RtlClearBits [0x7710BB1F+191]


<h2>Save data</h2>

Data saved in json file

In [ ]:
json_data = json.dumps(data, indent=4)
 
# Writing to sample.json
with open(targetIcao + "_" + save_file, "w") as target_file:
    target_file.write(json_data)

<h2>Load data</h2>

In [6]:
#Parameters
load_location = "./a835af_tracking_data_2020-01-19_2023-02-26.json"

In [9]:
with open(load_location, 'r') as f:
    json_data = json.load(f)

<h2>Clean Data</h2>

1. Get succeding locations where plane stopped

In [60]:
#Dependencies
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from math import sin, cos, acos, radians

In [58]:
def getDist(p1, p2):
    if p1 == None:
        return 6371
    l1, l2 = radians(p1[0]), radians(p2[0])
    return acos(sin(l1) * sin(l2) + cos(l1) * cos(l2) * cos(radians(p2[1]) - radians(p1[1]))) * 6371

In [89]:
#1. Get locations of plane
destinations_list = []

# Initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")
# reverse_f = RateLimiter(geolocator.reverse)

# Initialize progress bar
n = len(json_data)
c = 0
l_c = -1

toolbar_width = 40

#Run through data
print("Isolating locations")

last_loc = None

for date in json_data:
    entry = json_data[date]
    
    if (entry["baro_alt"] == "ground"):
        location = (entry["lat"] , entry["lon"])

        if getDist(last_loc, location) > 2: #location != last_loc:
            destinations_list.append([date, location])
            last_loc = location
    
    if ((c * 100) % n < 100):
        a = (c * toolbar_width) // n
        print("[{}{}] {}%".format('■'*a, ' '*(toolbar_width - a), (c * 100) // n), end='\r', file = sys.stdout, flush=True)

    c += 1

a = (c * toolbar_width) // n
print("[{}{}] {}%".format('■'*a, ' '*(toolbar_width - a), (c * 100) // n), end='\n', file = sys.stdout, flush=True)

print("\nGetting associated addresses - {} instances to check".format(len(destinations_list)))
n = len(destinations_list)
c = 0

hold = []
last_loc = None

for point in destinations_list:
    location = geolocator.reverse( point[1] )
    
    if(location != None):
            location = location.raw['address']
            road = location.get('road', '')
            city = location.get('city', '')
            if (len(city) < 1):
                city = location.get('town', '')
                if (len(city) < 1):
                    city = location.get('village', '')
            state = location.get('state', '')
            country = location.get('country', '')
            postcode = location.get('postcode', '')

            
            if(len(country) > 0):
                location_str = "{}, {}, {}, {}, {}".format(road, city, state, country, postcode)

                if location_str != last_loc:
                    hold.append([point[0], location_str])
                    last_loc = location_str
            else:
                print(location)
    
    if ((c * 100) % n < 100):
        a = (c * toolbar_width) // n
        print("[{}{}] {}%".format('■'*a, ' '*(toolbar_width - a), (c * 100) // n), end='\r', file = sys.stdout, flush=True)

    c += 1

a = (c * toolbar_width) // n
print("[{}{}] {}%".format('■'*a, ' '*(toolbar_width - a), (c * 100) // n), end='\n', file = sys.stdout, flush=True)

destinations_list = hold

Isolating locations
[■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100%

Getting associated addresses - 658 instances to check
[■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■] 100%


In [90]:
#Save landing locations to csv file
ext = load_location.split("/")[1].split(".")[0]
save_path = "landing_locations_{}.csv".format(ext)

with open(save_path, "w") as f:
    f.writelines(["{}, {}\n".format(v[0], v[1]) for v in destinations_list])

print("Saved to {}".format(save_path))

Saved to landing_locations_a835af_tracking_data_2020-01-19_2023-02-26.csv


In [10]:
#Load landing data
load_path = "landing_locations_a835af_tracking_data_2020-01-19_2023-02-26.csv"

destinations_list = []
with open(load_path, "r") as f:
    ls = f.readlines()
    for l in ls:
        l = l.split("\n")[0]
        l = l.split(",")
        destinations_list.append([l[0], "{}, {}, {}, {}, {}".format(l[1], l[2], l[3], l[4], l[5])])

<h2>Display data</h2>

1. Display trace of plane

In [11]:
import plotly.express as px
import plotly.graph_objects as go

In [16]:
#Display trace on screen
data_trace = {'lat':[], 'lon':[], 'timestamp':[], 'label':[]}

for location in destinations_list:
    entry = json_data[location[0]]
    data_trace['lat'].append(entry["lat"])
    data_trace['lon'].append(entry["lon"])
    data_trace['timestamp'].append(location[0])
    data_trace['label'].append(location[1])

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = data_trace['lon'],
    lat = data_trace['lat'],
    hoverinfo = 'text',
    text = data_trace['label'],
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

for i in range(len(data['lon']) - 1):
    fig.add_trace(
        go.Scattergeo(
            locationmode = 'USA-states',
            lon = [data_trace['lon'][i], data_trace['lon'][i + 1]],
            lat = [data_trace['lat'][i], data_trace['lat'][i + 1]],
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = 1#float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
        )
    )

fig.update_layout(
    showlegend = False,
    geo = dict(
        # scope = 'north america',
        projection_type = 'equirectangular',

        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',

        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'
    ),
)

fig.show()

NameError: name 'data' is not defined

2. Display unique location where plane stoped

In [12]:
#Get unique locations
unique_locs = {}

for data_point in destinations_list:
    point_str = data_point[1]

    if not point_str in unique_locs:
        entry = json_data[data_point[0]]
        unique_locs[point_str] = [1, [entry["lat"], entry["lon"]]]
    else:
        unique_locs[point_str][0] += 1

In [10]:
print(unique_locs)

{' Center Lane,  Titusville,  Florida,  United States,  32780': [14, [28.511867, -80.796692]], ' Sophia Avenue,  Los Angeles,  California,  United States,  91406': [20, [34.212077, -118.490567]], ' SpaceX Starlink Entrance,  Hawthorne,  California,  United States,  90250': [6, [33.922545, -118.341479]], ' Boring Test Tunnel,  Hawthorne,  California,  United States,  90250': [121, [33.922737, -118.337509]], ' Northside Parkway,  Los Angeles,  California,  United States,  90045': [4, [33.95074, -118.415451]], ' ,  Los Angeles,  California,  United States,  90245': [1, [33.93352, -118.40837]], ' International Road,  Los Angeles,  California,  United States,  90045': [2, [33.938381, -118.383052]], ' Airport Boulevard,  San Jose,  California,  United States,  95110': [62, [37.363311, -121.930593]], ' Perimeter Drive,  ,  Florida,  United States,  33142': [1, [25.804673, -80.291946]], ' Presidential Boulevard,  Austin,  Texas,  United States,  78719': [106, [30.18988, -97.657538]], ' Mathew 

In [23]:
#Display locations on screen
data_unique = {'lat':[], 'lon':[], 'data':[], 'label':[]}

for location in unique_locs:
    entry = unique_locs[location]
    data_unique['lat'].append(entry[1][0])
    data_unique['lon'].append(entry[1][1])
    data_unique['data'].append(entry[0])
    data_unique['label'].append(location)

df = pd.DataFrame(data_unique)

# create the scatter plot on a map
fig = px.scatter_geo(df, lat='lat', lon='lon', size = 'data', projection="natural earth")

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

# display the plot
fig.show()


3. Combine it all

In [21]:
#Make traces
data_trace = {'lat':[], 'lon':[], 'timestamp':[], 'label':[]}

for location in destinations_list:
    entry = json_data[location[0]]
    data_trace['lat'].append(entry["lat"])
    data_trace['lon'].append(entry["lon"])
    data_trace['timestamp'].append(location[0])
    data_trace['label'].append(location[1])

#Make marker sizes
min_d_s = min(data_unique['data'])
max_d_s = max(data_unique['data'])
min_p_s = 3.5
max_p_s = 15
marker_sizes = [(v - min_d_s)/(max_d_s - min_d_s) * (max_p_s - min_p_s) + min_p_s for v in data_unique['data']]

fig = go.Figure()
fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = data_unique['lon'],
    lat = data_unique['lat'],
    hoverinfo = 'text',
    text = data_unique['label'],
    mode = 'markers',
    marker = dict(
        size = marker_sizes,
        color = 'rgb(0, 0, 255)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

for i in range(len(data_trace['lon']) - 1):
    fig.add_trace(
        go.Scattergeo(
            # locationmode = 'USA-states',
            lon = [data_trace['lon'][i], data_trace['lon'][i + 1]],
            lat = [data_trace['lat'][i], data_trace['lat'][i + 1]],
            hoverinfo='skip',
            mode = 'lines',
            line = dict(width = 1,color = 'red'),
            opacity = 1#float(df_flight_paths['cnt'][i]) / float(df_flight_paths['cnt'].max()),
        )
    )

fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    showlegend = False,
    geo = dict(
        # scope = 'north america',
        projection_type = 'equirectangular',

        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',

        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)'
    ),
)

fig.show()

<h2>Making SQL Databas</h2>

1. Format json data to csv file
2. Make SQL database from reformated data

In [19]:
csv_save_file = "a835af_tracking_data_2020-01-19_2023-02-26.csv"

with open(csv_save_file, "w") as target_file:
    target_file.write("{}, {}, {}, {}, {}\n".format("Time", "latitude", "longitude", "barometric altitude", "ground speed"))
    for date in json_data:
        entry = json_data[date]
        target_file.write("{}, {}, {}, {}, {}\n".format(date, entry["lat"], entry["lon"], entry["baro_alt"], entry["grd_speed"]))
        

In [26]:
#Load csv
import pandas as pd

csv_trace_path = "a835af_tracking_data_2020-01-19_2023-02-26.csv"
csv_landing_path = "landing_locations_a835af_tracking_data_2020-01-19_2023-02-26.csv"

df_csv_trace = pd.read_csv(csv_trace_path, ',')
df_csv_landings = pd.read_csv(csv_landing_path, ',')

#Make database
import sqlite3

con =  sqlite3.connect('where_is_elon.db')

#Add to database
df_csv_trace.to_sql("trace", con=con)
df_csv_landings.to_sql("landings", con=con)

con.commit() #Just in case
con.close()

<h2>Test Elements - Ignore</h2>

In [181]:
indices = []
for i in range(len(list(data.keys()))):
    if(data[list(data.keys())[i]]['extra'] != None):
        indices.append(i)

print(data[list(data.keys())[indices[100]]])

{'lat': 32.164948, 'lon': -106.608966, 'baro_alt': 43000, 'grd_speed': 464, 'extra': {'flight_num': 'N628TS  ', 'geom_alt': 44000, 'baro_rate': -64, 'grd_track': 306.5}}


In [182]:
print(sorted(list(js_data[0].keys())))
print("---------------------------------------")
print(len(js_data[0]['fullTrace']['trace']))
print(js_data[0]['fullTrace']['trace'][100])
print("")
print(js_data[0]['fullTrace']['trace'][400])

['addrtype', 'alt_baro', 'alt_reliable', 'alt_rounded', 'altitude', 'altitudeTime', 'bad_alt', 'bad_altTime', 'baseMarkerKey', 'baseScale', 'bgColorCache', 'category', 'clearTraceAfter', 'country', 'dataSource', 'dbinfoLoaded', 'drawLine', 'elastic_feature', 'fakeHex', 'flag_image', 'flight', 'flightTs', 'fullTrace', 'geom_diff', 'geom_diff_ts', 'gs', 'history_size', 'icao', 'icaoType', 'icaoTypeCache', 'inView', 'interesting', 'ladd', 'last', 'lastTraceTs', 'lastVisible', 'last_info_server', 'last_message_time', 'layer', 'linesDrawn', 'marker', 'markerDrawn', 'markerIcon', 'markerStyle', 'markerStyleKey', 'markerSvgKey', 'messageRate', 'messageRateOld', 'messages', 'military', 'msgs1090', 'msgs978', 'nac_p', 'nac_v', 'name', 'nav_altitude', 'noInfoTime', 'numHex', 'onGround', 'ownOp', 'pia', 'position', 'position_time', 'prev_alt', 'prev_alt_rounded', 'prev_dataSource', 'prev_rot', 'prev_speed', 'prev_time', 'proj', 'psAPIparam', 'psAPIresponse', 'psAPIresponseTS', 'rc', 'recentTrace'

In [183]:
print(datetime.strptime("1970/01/01", '%Y/%m/%d') + timedelta(seconds=1675820888))

2023-02-08 01:48:08


In [184]:
# from selenium.webdriver.chrome.service import Service
# from time import sleep

# chromedriver = "C:/webdrivers/chromedriver.exe"

# option = webdriver.ChromeOptions()
# option.binary_location = "C:/Program Files/BraveSoftware/Brave-Browser/Application/brave.exe"

# s = Service(chromedriver)

# driver = webdriver.Chrome(service=s, options=option)

# #Patch Fingerprint leaks
# script = "Object.defineProperty(navigator, 'webdriver', {get: () => false})"
# driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {"source": script})

# delta = endDate - startDate
# for i in range(delta.days + 1):
#     driver.get("https://globe.adsbexchange.com/?icao={}&lat={}&lon={}&zoom={}&showTrace={}".format(targetIcao, lat, lon, zoom, startDate + timedelta(days=i)))
#     # sleep(10)
#     # element = driver.execute_script("return selectedPlanes()")
#     # print(element)
# # driver.close()